In [1]:
import numpy as np
import pdfplumber
import os
import re
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, PatternFill, Border, Alignment
from pathlib import Path

## Step 1: Fill Excel "DoesCal.xlsx" (Manual) :
    In Sheet "Initial", fill all cells in yellow with fixed units.

## Step 2: Save PACE4 results in Folder "PACE4" (Manual):
    Make sure the beam energy for PACE4 is same as the one in Excel Sheet "Initial"

## Step 3: Run the code to copy info from PACE4 result to "DoesCal.xlsx"

### Code(Extract_CompoundInf): 

In [2]:
def Extract_CompoundInf(text):
    # Return Values:
    Coulomb_Barrier = -1
    Center_Mass = -1
    Recoil_Energy = -1
    Beam_Energy = -1
    Compound_A = -1
    
    lines = text.split('\n')
    # loop all lines in this text(page)
    for i, line in enumerate(lines):
        if "Bass fusion xsection for E = " in line:
            match = re.search(r'Bass fusion xsection for E = (\d+(\.\d+)?)\s*MeV', line)
            if match:
                Beam_Energy = float(match.group(1))
                continue
        if "Barrier height" in line:
            match = re.search(r'Barrier height is (\d+(\.\d+)?)\s*MeV', line)
            if match:
                Coulomb_Barrier = float(match.group(1))
                continue
        if "Center of mass energy (MeV)" in line:
            match = re.search(r'Center of mass energy \(MeV\)\s*(\d+(\.\d+)?)', line)
            if match:
                Center_Mass = float(match.group(1))
                continue
        if "Compound nucleus recoil energy (MeV)" in line:
            match = re.search(r'Compound nucleus recoil energy \(MeV\)\s*(\d+(\.\d+)?)', line)
            if match:
                Recoil_Energy = float(match.group(1))
                break
        if "Compound nucleus " in line and Compound_A < 0:
            columns = line.split()
            Compound_A = float(columns[-1])

    if Beam_Energy<0:
        print("Beam energy NOT Found")
    if Coulomb_Barrier<0:
        print("Barrier height NOT Found")
    if Center_Mass<0:
        print("Center of mass energy NOT Found")
    if Recoil_Energy<0:
        print("Compound nucleus recoil energy NOT Found")
    if Compound_A<0:
        print("Compound nucleus info NOT Found")
    
        
    return Beam_Energy, Coulomb_Barrier, Center_Mass, Recoil_Energy, Compound_A

### Code(Extract_ProductInf): extract Production Info from PDF

In [3]:
def Extract_ProductInf(pdf):
    start_flag = False 
    end_flag = False
    Production_Inf = []
    for page_num, page in enumerate(pdf.pages[1:]):
        if start_flag and end_flag: break
        text = page.extract_text()
        if text:
            lines = text.split('\n')
            for i, line in enumerate(lines):
                if "Yields of residual nuclei" in line: 
                    start_flag = True
                    continue
                if "Angular distribution results" in line: 
                    end_flag = True
                    break
                if start_flag and not end_flag:
                    Production_Inf.append(line.split())
    if not Production_Inf:
        print("Yields of residual nuclei NOT Found")
    return Production_Inf

### Code(Filter_Inf): filter Production Info

In [4]:
def Filter_Inf(Production):
    # If production cross pages, there will be line written in "Production" with one element which returns page number
    # So we need to delete it first
    Production = [row for row in Production if len(row) > 1]

    # Take out the last line which contains total information for the reaction
    # Then delete it from "Production"
    Total = Production[-1]
    Production = Production[:-1]

    # Add "isotop for the Column name"
    Production[0].insert(3, "isotope")
    
    # Only Record Z, N, A, isotope, xsection information into Excel
    # And convert all values to int or float
    Production_Record = [[row[0], row[1], row[2], row[3], row[6]] for row in Production]
    for row in Production_Record[1:]:
        row[0] = int(row[0])  # Convert Z to int
        row[1] = int(row[1])  # Convert N to int
        row[2] = int(row[2])  # Convert A to int
        row[4] = float(row[4])  # Convert x-section(mb) to float
    
    return Total, Production_Record

### Code(Write_Excel): write info to excel

In [5]:
def Write_Excel(Production_Record, Total, file_path, Beam_Energy=0, Recoil_Energy=0, Compound_A = 0):
    #================== Update "Initial" sheet ================#
    # Load the existing workbook and the "Initial" sheet
    workbook = load_workbook(file_path)
    existing_df = pd.read_excel(file_path, sheet_name="Initial", engine='openpyxl')
    
    # Update the existing DataFrame with recoil energy and total x-sec for the current beam energy
    row_index = existing_df[existing_df["Beam Energy (MeV)"] == Beam_Energy].index[0]
    existing_df.at[row_index, "Recoil Energy (MeV)"] = Recoil_Energy
    existing_df.at[row_index, "Total xsec (mb)"] = float(Total[-1])
    thickness = existing_df.at[row_index,"Target layer (um)"]

    # Write the updated DataFrame back to the "Initial" sheet
    existing_worksheet = workbook["Initial"] 
    for row in range(len(existing_df)):
        for col in range(len(existing_df.columns)):
            existing_worksheet.cell(row=row + 2, column=col + 1, value=existing_df.iat[row, col])  # +2 to account for header
    # Save the workbook after updating the existing sheet
    workbook.save(file_path)

    #================== Create new sheet for PACE4 result ================#
    # Convert the first row as headers and the rest recorded into DataFrame
    df = pd.DataFrame(Production_Record[1:], columns=Production_Record[0])
    
    # Create a new row for the DataFrame based on Total
    total_row = pd.DataFrame({
        'Z': [Total[0]],  # The first element of Total in 'Z' column
        'x-section(mb)': [float(Total[-1])]  # The last element of Total in 'x-section(mb)' column
    })
    
    # Append the total_row to the DataFrame
    df = pd.concat([df, total_row], ignore_index=True)

    # Insert a new column "Recoil Energy(MeV)" after "x-section(mb)"
    df.insert(df.columns.get_loc('x-section(mb)') + 1, 'Recoil_E(MeV)', 
              df['A'].apply(lambda A: Recoil_Energy / Compound_A * A))
    
    # Create a new sheet name based on Beam Energy
    new_sheet_name = f'{int(thickness)}um_{int(Beam_Energy)}MeV'

    # Now create the new sheet with the updated DataFrame
    with pd.ExcelWriter(file_path, mode='a', engine='openpyxl', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=new_sheet_name, index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(file_path)

    # Set column width and alignment for the new sheet
    worksheet = workbook[new_sheet_name]
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 16
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
   
    # Final save
    workbook.save(file_path)

### Code(RWProcess): Read PDF and Write EXCEL

In [6]:
def RWProcess(pdf_path, excel_path):
    with pdfplumber.open(pdf_path) as pdf:
        # Extract basic information for this reaction
        page = pdf.pages[0]
        text = page.extract_text()
        Beam_Energy, Coulomb_Barrier, Center_Mass, Recoil_Energy, Compound_A = Extract_CompoundInf(text)
        # Extract info for productions from the reaction
        Production = Extract_ProductInf(pdf)

    # Filter Production info to write them into Spreadsheet
    Total, Production_Record = Filter_Inf(Production)

    # Write Production Info into Excel:
    Write_Excel(Production_Record, Total, excel_path, Beam_Energy, Recoil_Energy, Compound_A)    

### Code(RWPACE4): loop PACE4 folder

In [7]:
def RWPACE4(folder_path, excel_path):
    for pdf_file in os.listdir(folder_path):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(folder_path, pdf_file)
            RWProcess(pdf_path, excel_path)

## Step 4: Extract Half-Lives
### 4.1 Read half-life information from NNDC :
1. Go to Nuclear Levels and Gammas Search;
2. Give the range for Z or A or N to search the result (I usually set half-live range > 1 sec);
3. Save the result page as PDF (Right click -> choose print -> Save as PDF);

* No space in any path

### 4.2 Extract half-lives:
1. Only write the half-life info into the .txt file same name as .pdf file (ONE TIME Operation!!!! Unless you want to update the old one)
2. Extract the half-lives from .txt file, which is faster than reading .pdf file;
3. Add half-life info to isotopes saved in the Excel sheet created in Step 3;

### Code(Convert_PDF_to_TXT): convert t1/2 PDF to TXT

In [2]:
def Convert_PDF_to_TXT(pdf_path, flag_overwrite = False):
    # Get the base name of the PDF file (without the extension) using string operations
    base_name = pdf_path.split('/')[-1].replace('.pdf', '')
    # Create the output TXT file name
    txt_path = pdf_path.replace('.pdf', '.txt')
    
    # if txt file exists and don't need to be replaced, return empty
    if os.path.exists(txt_path) and flag_overwrite:
        return
    # if txt file doesn't exist or you want to overwrite it
    else:
        # Open the PDF file
        with pdfplumber.open(pdf_path) as pdf:
            text_to_write = []
            capture_text = False # will be "True" when detect "Nucleus E level(keV) Jπ T 1/2" where start extracting info
            # Iterate through each page
            for page in pdf.pages:
                # Extract the text from the page
                text = page.extract_text() 
                # Split the text into lines
                lines = text.split('\n')
                for line in lines:
                    # Skip the unwanted header and footer lines
                    #if "page" in line.lower() and "," in line and ("AM" in line or "PM" in line):
                    if "AM" in line or "PM" in line:
                        continue
                    if "https://www.nndc.bnl.gov" in line:
                        continue     
                    # Start capturing text after the target start line
                    if "Nucleus E level(keV) Jπ T 1/2" in line:
                        capture_text = True
                        continue
                    # Stop capturing text after the target end line
                    if "Gamma Information" in line:
                        capture_text = False
                        continue
                    if "≥" in line or ">" in line:
                        continue
                    if "eV" in line:
                        continue
                    # Capture the text
                    if capture_text:
                        text_to_write.append(line)
        # Write the captured text to the dynamically named txt file
        with open(txt_path, "w") as output_file:
            output_file.write("\n".join(text_to_write))

### mini Code(parse_t_half): parse t1/2 info in txt file

In [9]:
# ============= Function to parse the t_half value into t_value + t_unit ==============#
# the input "t_half" should have one character (no-digit) part
def parse_t_half(t_half):
    # if t1/2 = stable, will return the age of the universe
    if t_half.upper() == "STABLE":
        return 1.4e10, "y"
    matches = re.findall(r'(\d+(?:\.\d+)?(?:e[-+]?\d+)?)\s*([a-zA-Z]+)', t_half)
    # Initialize variables for the extracted values
    t_value = -1
    t_unit = None
    # If matches are found, assign the number and unit
    if matches:
        t_value = float(matches[0][0])  # First number found
        t_unit = matches[0][1]    # Corresponding unit
    return t_value, t_unit

### Code(Extract_HalfLife): Extract A, nucleus_name, t1/2_value and t1/2_unit into an array

In [10]:
# ============= Function to extract t1/2 from txt ==============#
# extracted_data = [A,"nucleus_name", t1/2, "unit"]
def Extract_HalfLife(txt_path):
    # Open the existing TXT file and process it
    with open(txt_path, "r") as input_file:
        extracted_data = []
        # Skip the first line (header)
        next(input_file)
        for line in input_file:
            # Skip any empty lines
            if not line.strip():
                continue
            # Split the line into columns
            columns = line.split()
            # Extract Nucleus (separated into number and letters, like "1H" to "1" and "H")
            nucleus_full = columns[0]
            nucleus_number = ''.join([char for char in nucleus_full if char.isdigit()])
            nucleus_letters = ''.join([char for char in nucleus_full if char.isalpha()])
            # Extract and parse T 1/2
            t_half = ' '.join(columns[3:])
            value, unit = parse_t_half(t_half)
            if value > 0:
                # Append the extracted data as a tuple
                extracted_data.append((float(nucleus_number), nucleus_letters, value, unit))
        return extracted_data

### mini Code(HalfLife_Unit_Factor): scaling factor to convert t1/2 in seconds

In [11]:
#============= convert all half-lives to seconds =============#
def HalfLife_Unit_Factor(unit):
    fac = 0
    if unit == "s":
        fac = 1;
    if unit == "m":
        fac = 60;
    if unit == "h":
        fac = 60*60;
    if unit == "d":
        fac = 24*60*60;
    if unit == "y":
        fac = 365*24*60*60;
    return fac

### mini Code(Fill_HalfLife_Sheet): Fill t1/2 info into one Sheet

In [12]:
#============= Fill Half-Life to one sheet in Excel =============#
def Fill_HalfLife_Sheet(txt_path, excel_path, sheet_name):
    sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets
    df = sheets[sheet_name]
    # Add new columns for "t1/2" and "unit" if they don't already exist
    if "t1/2" not in df.columns:
        df["t1/2"] = -1.0
    df.at[df.index[-1], "t1/2"] = np.nan
    if "unit" not in df.columns:
        df["unit"] = ""
    if "t1/2(s)" not in df.columns:
        df["t1/2(s)"] = -1.0
    df.at[df.index[-1], "t1/2(s)"] = np.nan
    # Iterate over each entry in the extracted_data array

    extracted_data = Extract_HalfLife(txt_path);
    for data in extracted_data:
        A = data[0]       # A
        isotope = data[1] # "Isotope"
        t_value = data[2] # t1/2
        t_unit = data[3]  # "unit"
        # Loop over all rows in the DataFrame to check for matches
        for idx in range(len(df)-1):
            if df.at[idx, "A"] == A and df.at[idx, "isotope"].lower() == isotope.lower():
                # Fill the first matching row with t1/2 and unit data
                if df.at[idx, "t1/2"] < 0:
                    df.at[idx, "t1/2"] = t_value
                    df.at[idx, "unit"] = t_unit
                    df.at[idx, "t1/2(s)"] = t_value * HalfLife_Unit_Factor(t_unit)
                    break
                else:
                    # If there are already values, create a new row
                    new_row = df.loc[idx].copy()
                    new_row["t1/2"] = t_value
                    new_row["unit"] = t_unit
                    new_row["t1/2(s)"] = t_value * HalfLife_Unit_Factor(t_unit)
                    # Append " isomer" to the Isotope in the new row
                    new_row["isotope"] = f"{new_row['isotope']}_isomer"
                    df = pd.concat([df.iloc[:idx + 1], pd.DataFrame([new_row]), df.iloc[idx + 1:]]).reset_index(drop=True)
                    break
    # Save df back to Excel
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)
    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook[sheet_name]
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 16
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
    # Save the workbook
    workbook.save(excel_path)

### Code(Fill_HalfLife_Exce): Fill t1/2 info to all sheets in Exxcel

In [13]:
#========================== Fill Half-Life to all sheets ("MeV") in Excel =============================#
def Fill_HalfLife_Excel(txt_path, excel_path):
    # To check if txt file exists
    if os.path.exists(txt_path) is False:
        # Get the base name of the txt file (without the extension) using string operations
        base_name = txt_path.split('/')[-1].replace('.txt', '')
        # Create the output TXT file name
        pdf_path = txt_path.replace('.txt', '.pdf')
        Convert_PDF_to_TXT(pdf_path)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames   
    # Iterate over all sheets and call Fill_HalfLife_Sheet() if the sheet name includes "MeV"
    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            Fill_HalfLife_Sheet(txt_path, excel_path, sheet_name)

## Step 5: Calculate Production Rate
---
1. Read information from Sheet "Initial"
2. Calculate production rates in each sub Sheet

* Be really really careful on unit, the current code is based on unit:
  1. thickness: mg/cm2;
  2. molar mass: g;
  3. x-section: mb;
  4. beam rate: pps;

### Code(ReadInitial): Read Sheet "Initial":
1. thickness in "mg/cm2";
2. molar mass in "g"
3. beam rate in "pps"

In [14]:
def ReadInitial(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)  # Load all sheets
    df = sheets["Initial"]
    
    # locate the target info
    target_col_index = df.columns.get_loc('Target')
    thick_row_idx = df[df.iloc[:, target_col_index + 2] == 'mg/cm2'].index[0]
    thick_tot = df.iloc[thick_row_idx, target_col_index + 1]
    mass_row_idx = df[df.iloc[:, target_col_index + 2] == 'g'].index[0]
    mass = df.iloc[mass_row_idx, target_col_index + 1]

    # locate the beam info
    beam_col_index = df.columns.get_loc('Beam')
    rate_row_idx = df[df.iloc[:, beam_col_index + 2] == 'pps'].index[0]
    rate = df.iloc[rate_row_idx, beam_col_index + 1]

    return thick_tot, mass, rate

### Code(Cal_Prodx_Rate): Calculate production rate in on Sheet

In [15]:
def Cal_Prodx_Rate(df, NLayer, thick_tot, mass, rate):
    if "Prodx_Rate(pps)" not in df.columns:
        df["Prodx_Rate(pps)"] = -1.0
    df.at[df.index[-1], "Prodx_Rate(pps)"] = np.nan
    for idx in range(len(df)-1):
        xsec = df.at[idx,'x-section(mb)']
        # Eqn: prodx_rate = xsec(m2) * unit_thick(g/m2) / mass(g) *rate (atom/s)
        # 1 mb = 1e-3 barn = 1e-3 * 1e-28 m2
        # 1 mg/cm2 = 10 g/m2
        df.at[idx,'Prodx_Rate(pps)'] = (xsec/1e3/1e28)*((thick_tot/NLayer)*10)/mass*rate
    return df

### Code(Write_Prodx_Rate): Update all sheets in Excel

In [16]:
def Write_Prodx_Rate(excel_path, NLayer):
    thick_tot, mass, rate = ReadInitial(excel_path)

    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames   
    # Iterate over all sheets and call Fill_HalfLife_Sheet() if the sheet name includes "MeV"
    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            df = Cal_Prodx_Rate(df, NLayer, thick_tot, mass, rate)
            # Save df back to Excel
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
                for cell in column:  # Align each cell in the column
                    cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

## Step 6: Generate Stopping Ratio
---
1. Currently, set stopping ratios for all isotopes under all cases are 1, which means all productions will stop in the target;
2. Generate values and save into Excel and create txt foils (optional, named with sub sheet name):

+ ToDo:
Either SRIM results or GEANT4 codes for thses values, based on recoil energy and the rest of thickness

### Code(Generate_Stop_Ratio):
+ Default:
+ Generate stopping ratio of all isotopes under all cases = 1;</br>
+ Only update Excel

In [17]:
def Generate_Stop_Ratio(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames
    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            if "Stop_Ratio" not in df.columns:
                df["Stop_Ratio"] = 1
            else: 
                print("Stopping Ratio has been generated")
                return 
            df.at[df.index[-1], "Stop_Ratio"] = np.nan
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
            for cell in column:  # Align each cell in the column
                cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

## Step 7: Calculate Production A(Bq) and Summarize Sub Sheet

### 7.1: Calculate Production A(Bq) at cd = 0:
A = Prodx_Rate * (1-exp(-0.693/halflife * beam_time)) * Stop_Ratio
+ Everything in the equation above is in seconds

### Code(Cal_Prodx_A)

In [18]:
def Cal_Prodx_A(excel_path):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)
    sheet_names = workbook.sheetnames

    df = sheets["Initial"]
    beam_col_index = df.columns.get_loc('Beam')
    run_time_row_idx = df[df.iloc[:, beam_col_index + 2] == 'sec'].index[0]
    run_time = df.iloc[run_time_row_idx, beam_col_index + 1]

    for sheet_name in sheet_names:
        if "MeV" in sheet_name:
            df = sheets[sheet_name]
            if "Prodx_A(Bq)" not in df.columns:
                df["Prodx_A(Bq)"] = -1.0
            df.at[df.index[-1], "Prodx_A(Bq)"] = np.nan
            for idx in range(len(df)-1):
                prodx_rate = df.at[idx,'Prodx_Rate(pps)']
                halflife = df.at[idx,'t1/2(s)']
                stop_ratio = df.at[idx,'Stop_Ratio']
                df.at[idx,"Prodx_A(Bq)"] = prodx_rate * (1 - np.exp(-0.693/halflife * run_time)) * stop_ratio


            # Save df back to Excel
            with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                df.to_excel(writer, sheet_name=sheet_name, index=False)
            # Reopen the workbook to apply formatting
            workbook = load_workbook(excel_path)
            # Set column width and alignment for the new sheet
            worksheet = workbook[sheet_name]
            for column in worksheet.columns:
                column_letter = column[0].column_letter
                worksheet.column_dimensions[column_letter].width = 16
                for cell in column:  # Align each cell in the column
                    cell.alignment = Alignment(horizontal='right')
            # Save the workbook
            workbook.save(excel_path)

### 7.2 Summarize all sub sheets (including A, isotope, A(Bq) @ *um)
1. Summarize all production activities in one sheet;
2. Calculate sum activity of each production at cool down time = 0s;
3. (optional) Delete production row if A = 0 or t1/2 > 1e9 y (stable nuclei)

### Code(summarize)

In [19]:
def summarize(excel_path, clean_table = True):
    sheets = pd.read_excel(excel_path, sheet_name=None)
    workbook = load_workbook(excel_path)

    # Extract data from sheets named "*um_*MeV"
    data_frames = []
    for sheet_name in workbook.sheetnames:
        if 'um_' in sheet_name and 'MeV' in sheet_name:
            df = sheets[sheet_name][['A', 'isotope', 't1/2', 'unit', 'Prodx_A(Bq)']]
            df.columns = ['A', 'isotope', 't1/2', 'unit', f'A(Bq)@{sheet_name.split("um_")[0]}um']
            data_frames.append((int(sheet_name.split('um_')[0]), df))

    # Sort data_frames by the numeric value in the sheet names
    data_frames.sort(key=lambda x: x[0])

    # Extract only the DataFrames in the correct order
    data_frames = [df for _, df in data_frames]

    # Merge all data frames on 'A' and 'isotope'
    merged_df = data_frames[0]
    for df in data_frames[1:]:
        merged_df = pd.merge(merged_df, df, on=['A', 't1/2', 'unit', 'isotope'], how='outer')

    # Reorder by "A" from large to small, and then by "isotope" keyword
    merged_df['isotope_keyword'] = merged_df['isotope'].str.replace(' isomer', '')
    merged_df = merged_df.sort_values(by=['A', 'isotope_keyword', 'isotope'], ascending=[False, True, True])
    # Drop the temporary column used for sorting
    merged_df = merged_df.drop(columns=['isotope_keyword'])
    
    # Round all "Prodx" columns to 2 decimal places
    for col in merged_df.columns:
        if "um" in col:
            merged_df[col] = merged_df[col].fillna(0).round(2)

    # Add the new column "A(Bq)@cd = 0s" if it doesn't exist
    if "A(Bq)@cd = 0s" not in merged_df.columns:
        merged_df["A(Bq)@cd = 0s"] = -1

    # Calculate the sum of all "A(Bq)@*um" columns for each row
    sum_columns = [col for col in merged_df.columns if "um" in col]
    merged_df["A(Bq)@cd = 0s"] = merged_df[sum_columns].sum(axis=1)

    # Remove rows where "A(Bq)@cd = 0s" is 0
    if clean_table:
        merged_df = merged_df[merged_df["A(Bq)@cd = 0s"] != 0]
        merged_df = merged_df[merged_df["t1/2"] < 1e9]
    
    # Save the updated workbook
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists = 'replace') as writer:
        merged_df.to_excel(writer, sheet_name='Summary_Act', index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook['Summary_Act']
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)

### 7.3 Calculate A(Bq) with different cd time

### Code (Cal_A_cd)

In [20]:
def Cal_A_cd(excel_path, t):
    df = pd.read_excel(excel_path, sheet_name='Summary_Act')
    df['t1/2_seconds'] = df.apply(lambda row: row['t1/2'] * HalfLife_Unit_Factor(row['unit']), axis=1)
    match = re.match(r'(\d+\.?\d*)\s*([a-zA-Z]+)', t);
    t_value = float(match.group(1))
    t_unit = match.group(2)
    t_seconds = t_value * HalfLife_Unit_Factor(t_unit[0])
    df[f'A(Bq)@cd={t}'] = df.apply(
                                   lambda row: round(row['A(Bq)@cd = 0s'] * np.exp(-0.693 * t_seconds / row['t1/2_seconds']), 2) 
                                   if row['t1/2_seconds'] > 0 else 0, axis=1)
    
    df = df.drop(columns=['t1/2_seconds'])
    # Save the updated workbook
    with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists = 'replace') as writer:
        df.to_excel(writer, sheet_name='Summary_Act', index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook['Summary_Act']
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)

### Step 8: Read Does Parameters TXT file and Convert A(Bq) to uSv
* Take care unit!!!

#### Code(CalDoes)

In [21]:
def CalDoes(excel_path, doespar_path):
    df = pd.read_excel(excel_path, sheet_name='Summary_Act')
    doespar_df = pd.read_csv(doespar_path, sep='\\s+', names=['A', 'Isotope', 'DoesPar'], comment='#')

    # Filter columns containing "cd"
    cd_columns = [col for col in df.columns if "cd" in col]

    for idx, row in df.iterrows():
        # Get the current "A" and "Isotope" in the Excel sheet
        A = int(row['A'])
        isotope = row['isotope']
        match = doespar_df[(doespar_df['A'].astype(int) == A) & (doespar_df['Isotope'] == isotope)]
        if not match.empty:
            # Get the DoesPar value from the txt file
            does_par = match['DoesPar'].values[0]
            # Multiply DoesPar with each of the cd columns
            for col in cd_columns:
                new_col_name = f"Does(uSv)@{col[6:]}"  # Create new column name by adding Does(uSv)
                df.loc[idx, new_col_name] = row[col]/1e9 * does_par  # Calculate and assign new value
        else:
            df = df.drop(idx)

    # Sum all "Does(uSv)" columns and add a new row with the total sum
    does_columns = [col for col in df.columns if col.startswith("Does(uSv)")]  # Find all the new "Does(uSv)" columns
    sum_row = df[does_columns].sum()  # Calculate the sum of all the new "Does(uSv)" columns

    # Create a sum row and add it to the DataFrame
    sum_row_label = ['Total'] + [''] * (df.shape[1] - len(does_columns) - 1)  # Set 'Total' label for first column
    sum_row_full = pd.DataFrame([sum_row_label + sum_row.tolist()], columns=df.columns)

    # Append the sum row to the DataFrame
    df = pd.concat([df, sum_row_full], ignore_index=True)
    doespar_file = Path(doespar_path)
    sheet_name = f"{doespar_file.stem}"  # Create the new sheet name

    with pd.ExcelWriter(excel_path, mode='a', engine='openpyxl', if_sheet_exists = 'replace') as writer:
        df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Reopen the workbook to apply formatting
    workbook = load_workbook(excel_path)
    # Set column width and alignment for the new sheet
    worksheet = workbook[sheet_name]
    for column in worksheet.columns:
        column_letter = column[0].column_letter
        worksheet.column_dimensions[column_letter].width = 14
        for cell in column:  # Align each cell in the column
            cell.alignment = Alignment(horizontal='right')
            # Save the workbook
    workbook.save(excel_path)

## Test!!!

In [22]:
excel_path = "DoesCal.xlsx" # Initial Fill after Step 1
pace4_path = "PACE4/" # It is a folder containing all PDF of PACE4 results after Step 2

# Update PACE4 results to Excels (Codes in Step 3)
RWPACE4(pace4_path, excel_path)

t_path = "HalfLife/Levels_Results.txt" 
# Fill t1/2 info in to Excel (Codes in Step 4.2)
Fill_HalfLife_Excel(t_path, excel_path)

# Calculate production rates in sheet "..MeV"
# In this example, 15um target foil is separated by 5 layers. Each layer is 3um;
NLayer = 5
Write_Prodx_Rate(excel_path, NLayer) 

# Generate Stopping Ratio (Step 6)
Generate_Stop_Ratio(excel_path)

# Calculate Production Acitivity at cd = 0 s (Step 7.1)
Cal_Prodx_A(excel_path)

# Summarize Production info so far to a new sheet "Summary" (Step 7.2)
summarize(excel_path)

# Calculate A(Bq) after beam stop with different cd time (Step 7.3)
Cal_A_cd(excel_path, '15 m')
Cal_A_cd(excel_path, '1 hour')
Cal_A_cd(excel_path, '1 day')

# Convert A(Bq) to uSv
body_unshield = "DoesPar/Body_unshield.txt"
body_shield = "DoesPar/Body_shield.txt"
skin_unshield = "DoesPar/Skin_unshield.txt"
skin_shield = "DoesPar/Skin_shield.txt"
Breath = "DoesPar/Breath_Par.txt"
CalDoes(excel_path, body_unshield)
CalDoes(excel_path, body_shield)
CalDoes(excel_path, skin_unshield)
CalDoes(excel_path, skin_shield)
CalDoes(excel_path, Breath)

In [53]:
x = [[['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=0s'], [106, 'In', 6.2, 'm', 732.88], [106, 'In_m', 5.2, 'm', 733.53], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 128.17], [106, 'Ag_m', 8.28, 'd', 0.54], [105, 'In', 5.07, 'm', 1311.77], [105, 'In_m', 48.0, 's', 1312.13], [105, 'Cd', 55.5, 'm', 3049.34], [105, 'Ag', 41.29, 'd', 0.87], [105, 'Ag_m', 7.23, 'm', 1241.3], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 88.92], [104, 'In_m', 15.7, 's', 88.92], [104, 'Cd', 57.7, 'm', 479.86], [104, 'Ag', 69.2, 'm', 170.61], [104, 'Ag_m', 33.5, 'm', 268.55], [103, 'Ag', 65.7, 'm', 719.55], [103, 'Ag_m', 5.7, 's', 1534.43], [103, 'Pd', 16.99, 'd', 1.32], [103, 'Rh', 14000000000.0, 'y', 0.0], [103, 'Rh_m', 56.11, 'm', 46.54], [103, 'Ru', 39.25, 'd', 0.02], [102, 'Ag', 12.9, 'm', 2724.52], [102, 'Ag_m', 7.7, 'm', 2824.7], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.04], [102, 'Rh_m', 3.74, 'y', 0.01], [101, 'Ag', 11.1, 'm', 86.82], [101, 'Ag_m', 3.1, 's', 88.92], [101, 'Pd', 8.47, 'h', 15.76], [101, 'Rh', 3.3, 'y', 0.0], [101, 'Rh_m', 4.34, 'd', 0.15], [100, 'Rh', 20.5, 'h', 19.95], [100, 'Rh_m', 4.6, 'm', 599.96], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh', 16.1, 'd', 1.71], [99, 'Rh_m', 4.7, 'h', 131.07], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh', 8.72, 'm', 22.04], [98, 'Rh_m', 3.6, 'm', 22.23], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc', 6.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.01], [96, 'Tc', 4.28, 'd', 0.3], [96, 'Tc_m', 51.5, 'm', 24.63]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=1.5m'], [106, 'In', 6.2, 'm', 619.7527990880288], [106, 'In_m', 5.2, 'm', 600.6213185787035], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 122.72826695650605], [106, 'Ag_m', 8.28, 'd', 0.5399529232478276], [105, 'In', 5.07, 'm', 1068.5982944471589], [105, 'In_m', 48.0, 's', 357.82070966495974], [105, 'Cd', 55.5, 'm', 2992.758220739961], [105, 'Ag', 41.29, 'd', 0.8699847898822939], [105, 'Ag_m', 7.23, 'm', 1075.0673211343258], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 49.91078387337419], [104, 'In_m', 15.7, 's', 1.6738487643813924], [104, 'Cd', 57.7, 'm', 471.29244174818865], [104, 'Ag', 69.2, 'm', 168.06630492045178], [104, 'Ag_m', 33.5, 'm', 260.34489354421373], [103, 'Ag', 65.7, 'm', 708.2549295060942], [103, 'Ag_m', 5.7, 's', 0.02715508363458914], [103, 'Pd', 16.99, 'd', 1.31994391673], [103, 'Rh', 14000000000.0, 'y', 0.0], [103, 'Rh_m', 56.11, 'm', 45.6857324275925], [103, 'Ru', 39.25, 'd', 0.01999963216898763], [102, 'Ag', 12.9, 'm', 2513.587135283569], [102, 'Ag_m', 7.7, 'm', 2467.9853357451907], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.03999986070935975], [102, 'Rh_m', 3.74, 'y', 0.009999994711927265], [101, 'Ag', 11.1, 'm', 79.05852185700594], [101, 'Ag_m', 3.1, 's', 1.6265771539032085e-07], [101, 'Pd', 8.47, 'h', 15.72779658290452], [101, 'Rh', 3.3, 'y', 0.0], [101, 'Rh_m', 4.34, 'd', 0.14997505247805204], [100, 'Rh', 20.5, 'h', 19.93314693952193], [100, 'Rh_m', 4.6, 'm', 478.60935671347875], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh', 16.1, 'd', 1.7099233305231736], [99, 'Rh_m', 4.7, 'h', 130.58774305687157], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh', 8.72, 'm', 19.563200357626936], [98, 'Rh_m', 3.6, 'm', 16.654704481805645], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc', 6.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.00999992067339156], [96, 'Tc', 4.28, 'd', 0.29994940555185673], [96, 'Tc_m', 51.5, 'm', 24.137840301524626]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=15m'], [106, 'In', 6.2, 'm', 137.05318668696668], [106, 'In_m', 5.2, 'm', 99.36799053637195], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 83.05553061227786], [106, 'Ag_m', 8.28, 'd', 0.5395294171203965], [105, 'In', 5.07, 'm', 168.8206301580465], [105, 'In_m', 48.0, 's', 0.0029844406099864976], [105, 'Cd', 55.5, 'm', 2528.504133911394], [105, 'Ag', 41.29, 'd', 0.8698479107886412], [105, 'Ag_m', 7.23, 'm', 294.758141403612], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 0.2760252558070968], [104, 'In_m', 15.7, 's', 4.967833985567987e-16], [104, 'Cd', 57.7, 'm', 400.75025006801394], [104, 'Ag', 69.2, 'm', 146.81356065982624], [104, 'Ag_m', 33.5, 'm', 196.9083926955456], [103, 'Ag', 65.7, 'm', 614.2529650734187], [103, 'Ag_m', 5.7, 's', 4.623679463118311e-45], [103, 'Pd', 16.99, 'd', 1.3194392745151187], [103, 'Rh', 14000000000.0, 'y', 0.0], [103, 'Rh_m', 56.11, 'm', 38.6694956682147], [103, 'Ru', 39.25, 'd', 0.019996321994285575], [102, 'Ag', 12.9, 'm', 1217.1089243257286], [102, 'Ag_m', 7.7, 'm', 732.2759642464498], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.03999860711542437], [102, 'Rh_m', 3.74, 'y', 0.009999947119398481], [101, 'Ag', 11.1, 'm', 34.033676625297446], [101, 'Ag_m', 3.1, 's', 3.7303672956340844e-86], [101, 'Pd', 8.47, 'h', 15.440910900073655], [101, 'Rh', 3.3, 'y', 0.0], [101, 'Rh_m', 4.34, 'd', 0.14975071141139018], [100, 'Rh', 20.5, 'h', 19.782108613488422], [100, 'Rh_m', 4.6, 'm', 62.6197283383252], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh', 16.1, 'd', 1.709233459902945], [99, 'Rh_m', 4.7, 'h', 126.32650053842143], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh', 8.72, 'm', 6.691056796933273], [98, 'Rh_m', 3.6, 'm', 1.2385517183985255], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc', 6.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.009999206762232203], [96, 'Tc', 4.28, 'd', 0.29949443931566394], [96, 'Tc_m', 51.5, 'm', 20.128174539103377]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=2.5h'], [106, 'In', 6.2, 'm', 3.8335007145733046e-05], [106, 'In_m', 5.2, 'm', 1.526525807118809e-06], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 1.6734321656121036], [106, 'Ag_m', 8.28, 'd', 0.5353125824051731], [105, 'In', 5.07, 'm', 1.635068382021379e-06], [105, 'In_m', 48.0, 's', 4.862253782339166e-54], [105, 'Cd', 55.5, 'm', 468.580297516029], [105, 'Ag', 41.29, 'd', 0.8684803037665002], [105, 'Ag_m', 7.23, 'm', 0.00070756655339094], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 7.387425668980724e-24], [104, 'In_m', 15.7, 's', 2.6343632297822064e-171], [104, 'Cd', 57.7, 'm', 79.1966977333043], [104, 'Ag', 69.2, 'm', 37.98580813847746], [104, 'Ag_m', 33.5, 'm', 12.061903826388246], [103, 'Ag', 65.7, 'm', 147.88489516423502], [103, 'Ag_m', 5.7, 's', 0.0], [103, 'Pd', 16.99, 'd', 1.3144034516457397], [103, 'Rh', 14000000000.0, 'y', 0.0], [103, 'Rh_m', 56.11, 'm', 7.298736237983291], [103, 'Ru', 39.25, 'd', 0.01996325036531761], [102, 'Ag', 12.9, 'm', 0.8623517834741815], [102, 'Ag_m', 7.7, 'm', 0.0038725481313091298], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.039986073336684406], [102, 'Rh_m', 3.74, 'y', 0.009999471206568241], [101, 'Ag', 11.1, 'm', 0.007438932995588576], [101, 'Ag_m', 3.1, 's', 0.0], [101, 'Pd', 8.47, 'h', 12.844678870310881], [101, 'Rh', 3.3, 'y', 0.0], [101, 'Rh_m', 4.34, 'd', 0.1475256751699028], [100, 'Rh', 20.5, 'h', 18.333260947051503], [100, 'Rh_m', 4.6, 'm', 9.204731481528964e-08], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh', 16.1, 'd', 1.7023500432895249], [99, 'Rh_m', 4.7, 'h', 90.65984269072628], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh', 8.72, 'm', 0.00014656924554411633], [98, 'Rh_m', 3.6, 'm', 6.407465813826897e-12], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc', 6.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.00999207045324086], [96, 'Tc', 4.28, 'd', 0.29498256011512225], [96, 'Tc_m', 51.5, 'm', 3.2723834728944827]]]

In [55]:
x[1]

[['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=1.5m'],
 [106, 'In', 6.2, 'm', 619.7527990880288],
 [106, 'In_m', 5.2, 'm', 600.6213185787035],
 [106, 'Cd', 14000000000.0, 'y', 0.0],
 [106, 'Ag', 23.96, 'm', 122.72826695650605],
 [106, 'Ag_m', 8.28, 'd', 0.5399529232478276],
 [105, 'In', 5.07, 'm', 1068.5982944471589],
 [105, 'In_m', 48.0, 's', 357.82070966495974],
 [105, 'Cd', 55.5, 'm', 2992.758220739961],
 [105, 'Ag', 41.29, 'd', 0.8699847898822939],
 [105, 'Ag_m', 7.23, 'm', 1075.0673211343258],
 [105, 'Pd', 14000000000.0, 'y', 0.0],
 [104, 'In', 1.8, 'm', 49.91078387337419],
 [104, 'In_m', 15.7, 's', 1.6738487643813924],
 [104, 'Cd', 57.7, 'm', 471.29244174818865],
 [104, 'Ag', 69.2, 'm', 168.06630492045178],
 [104, 'Ag_m', 33.5, 'm', 260.34489354421373],
 [103, 'Ag', 65.7, 'm', 708.2549295060942],
 [103, 'Ag_m', 5.7, 's', 0.02715508363458914],
 [103, 'Pd', 16.99, 'd', 1.31994391673],
 [103, 'Rh', 14000000000.0, 'y', 0.0],
 [103, 'Rh_m', 56.11, 'm', 45.6857324275925],
 [103, 'Ru

In [56]:
y = [[['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=0s'], [106, 'In_m', 5.2, 'm', 733.53], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 128.17], [105, 'In_m', 48.0, 's', 1312.13], [105, 'Cd', 55.5, 'm', 3049.34], [105, 'Ag_m', 7.23, 'm', 1241.3], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 88.92], [104, 'Cd', 57.7, 'm', 479.86], [104, 'Ag_m', 33.5, 'm', 268.55], [103, 'Ag_m', 5.7, 's', 1534.43], [103, 'Pd', 16.99, 'd', 1.32], [103, 'Rh_m', 56.11, 'm', 46.54], [103, 'Ru', 39.25, 'd', 0.02], [102, 'Ag_m', 7.7, 'm', 2824.7], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.04], [101, 'Ag_m', 3.1, 's', 88.92], [101, 'Pd', 8.47, 'h', 15.76], [101, 'Rh_m', 4.34, 'd', 0.15], [100, 'Rh_m', 4.6, 'm', 599.96], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh_m', 4.7, 'h', 131.07], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh_m', 3.6, 'm', 22.23], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.01], [96, 'Tc_m', 51.5, 'm', 24.63]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=1.5m'], [106, 'In', 6.2, 'm', 619.7527990880288], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 122.72826695650605], [105, 'In', 5.07, 'm', 1068.5982944471589], [105, 'Cd', 55.5, 'm', 2992.758220739961], [105, 'Ag_m', 7.23, 'm', 1075.0673211343258], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 49.91078387337419], [104, 'Cd', 57.7, 'm', 471.29244174818865], [104, 'Ag_m', 33.5, 'm', 260.34489354421373], [103, 'Ag', 65.7, 'm', 708.2549295060942], [103, 'Pd', 16.99, 'd', 1.31994391673], [103, 'Rh_m', 56.11, 'm', 45.6857324275925], [103, 'Ru', 39.25, 'd', 0.01999963216898763], [102, 'Ag', 12.9, 'm', 2513.587135283569], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.03999986070935975], [101, 'Ag', 11.1, 'm', 79.05852185700594], [101, 'Pd', 8.47, 'h', 15.72779658290452], [101, 'Rh_m', 4.34, 'd', 0.14997505247805204], [100, 'Rh_m', 4.6, 'm', 478.60935671347875], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh_m', 4.7, 'h', 130.58774305687157], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh', 8.72, 'm', 19.563200357626936], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.00999992067339156], [96, 'Tc_m', 51.5, 'm', 24.137840301524626]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=15m'], [106, 'In', 6.2, 'm', 137.05318668696668], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 83.05553061227786], [105, 'In', 5.07, 'm', 168.8206301580465], [105, 'Cd', 55.5, 'm', 2528.504133911394], [105, 'Ag_m', 7.23, 'm', 294.758141403612], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 0.2760252558070968], [104, 'Cd', 57.7, 'm', 400.75025006801394], [104, 'Ag_m', 33.5, 'm', 196.9083926955456], [103, 'Ag', 65.7, 'm', 614.2529650734187], [103, 'Pd', 16.99, 'd', 1.3194392745151187], [103, 'Rh_m', 56.11, 'm', 38.6694956682147], [103, 'Ru', 39.25, 'd', 0.019996321994285575], [102, 'Ag', 12.9, 'm', 1217.1089243257286], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.03999860711542437], [101, 'Ag', 11.1, 'm', 34.033676625297446], [101, 'Pd', 8.47, 'h', 15.440910900073655], [101, 'Rh_m', 4.34, 'd', 0.14975071141139018], [100, 'Rh_m', 4.6, 'm', 62.6197283383252], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh_m', 4.7, 'h', 126.32650053842143], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh', 8.72, 'm', 6.691056796933273], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.009999206762232203], [96, 'Tc_m', 51.5, 'm', 20.128174539103377]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=2.5h'], [106, 'In', 6.2, 'm', 3.8335007145733046e-05], [106, 'Cd', 14000000000.0, 'y', 0.0], [106, 'Ag', 23.96, 'm', 1.6734321656121036], [105, 'In', 5.07, 'm', 1.635068382021379e-06], [105, 'Cd', 55.5, 'm', 468.580297516029], [105, 'Ag', 41.29, 'd', 0.8684803037665002], [105, 'Pd', 14000000000.0, 'y', 0.0], [104, 'In', 1.8, 'm', 7.387425668980724e-24], [104, 'Cd', 57.7, 'm', 79.1966977333043], [104, 'Ag', 69.2, 'm', 37.98580813847746], [103, 'Ag', 65.7, 'm', 147.88489516423502], [103, 'Pd', 16.99, 'd', 1.3144034516457397], [103, 'Rh_m', 56.11, 'm', 7.298736237983291], [103, 'Ru', 39.25, 'd', 0.01996325036531761], [102, 'Ag', 12.9, 'm', 0.8623517834741815], [102, 'Pd', 14000000000.0, 'y', 0.0], [102, 'Rh', 207.3, 'd', 0.039986073336684406], [101, 'Ag', 11.1, 'm', 0.007438932995588576], [101, 'Pd', 8.47, 'h', 12.844678870310881], [101, 'Rh_m', 4.34, 'd', 0.1475256751699028], [100, 'Rh', 20.5, 'h', 18.333260947051503], [100, 'Ru', 14000000000.0, 'y', 0.0], [99, 'Rh_m', 4.7, 'h', 90.65984269072628], [99, 'Ru', 14000000000.0, 'y', 0.0], [98, 'Rh', 8.72, 'm', 0.00014656924554411633], [98, 'Ru', 14000000000.0, 'y', 0.0], [97, 'Tc_m', 91.0, 'd', 0.00999207045324086], [96, 'Tc_m', 51.5, 'm', 3.2723834728944827]]]

In [57]:
y[0]

[['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=0s'],
 [106, 'In_m', 5.2, 'm', 733.53],
 [106, 'Cd', 14000000000.0, 'y', 0.0],
 [106, 'Ag', 23.96, 'm', 128.17],
 [105, 'In_m', 48.0, 's', 1312.13],
 [105, 'Cd', 55.5, 'm', 3049.34],
 [105, 'Ag_m', 7.23, 'm', 1241.3],
 [105, 'Pd', 14000000000.0, 'y', 0.0],
 [104, 'In', 1.8, 'm', 88.92],
 [104, 'Cd', 57.7, 'm', 479.86],
 [104, 'Ag_m', 33.5, 'm', 268.55],
 [103, 'Ag_m', 5.7, 's', 1534.43],
 [103, 'Pd', 16.99, 'd', 1.32],
 [103, 'Rh_m', 56.11, 'm', 46.54],
 [103, 'Ru', 39.25, 'd', 0.02],
 [102, 'Ag_m', 7.7, 'm', 2824.7],
 [102, 'Pd', 14000000000.0, 'y', 0.0],
 [102, 'Rh', 207.3, 'd', 0.04],
 [101, 'Ag_m', 3.1, 's', 88.92],
 [101, 'Pd', 8.47, 'h', 15.76],
 [101, 'Rh_m', 4.34, 'd', 0.15],
 [100, 'Rh_m', 4.6, 'm', 599.96],
 [100, 'Ru', 14000000000.0, 'y', 0.0],
 [99, 'Rh_m', 4.7, 'h', 131.07],
 [99, 'Ru', 14000000000.0, 'y', 0.0],
 [98, 'Rh_m', 3.6, 'm', 22.23],
 [98, 'Ru', 14000000000.0, 'y', 0.0],
 [97, 'Tc_m', 91.0, 'd', 0.01],
 [96, 'Tc_m

In [59]:
z = [[['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=0s'], [106, 'In_m', 5.2, 'm', 733.53], [106, 'Ag', 23.96, 'm', 128.17], [105, 'In_m', 48.0, 's', 1312.13], [105, 'Cd', 55.5, 'm', 3049.34], [105, 'Ag_m', 7.23, 'm', 1241.3], [104, 'In', 1.8, 'm', 88.92], [104, 'Cd', 57.7, 'm', 479.86], [104, 'Ag_m', 33.5, 'm', 268.55], [103, 'Ag_m', 5.7, 's', 1534.43], [103, 'Pd', 16.99, 'd', 1.32], [103, 'Rh_m', 56.11, 'm', 46.54], [103, 'Ru', 39.25, 'd', 0.02], [102, 'Ag_m', 7.7, 'm', 2824.7], [102, 'Rh', 207.3, 'd', 0.04], [101, 'Ag_m', 3.1, 's', 88.92], [101, 'Pd', 8.47, 'h', 15.76], [101, 'Rh_m', 4.34, 'd', 0.15], [100, 'Rh_m', 4.6, 'm', 599.96], [99, 'Rh_m', 4.7, 'h', 131.07], [98, 'Rh_m', 3.6, 'm', 22.23], [97, 'Tc_m', 91.0, 'd', 0.01], [96, 'Tc_m', 51.5, 'm', 24.63]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=1.5m'], [106, 'In', 6.2, 'm', 619.7527990880288], [106, 'Ag', 23.96, 'm', 122.72826695650605], [105, 'In', 5.07, 'm', 1068.5982944471589], [105, 'Cd', 55.5, 'm', 2992.758220739961], [105, 'Ag_m', 7.23, 'm', 1075.0673211343258], [104, 'In', 1.8, 'm', 49.91078387337419], [104, 'Cd', 57.7, 'm', 471.29244174818865], [104, 'Ag_m', 33.5, 'm', 260.34489354421373], [103, 'Ag', 65.7, 'm', 708.2549295060942], [103, 'Pd', 16.99, 'd', 1.31994391673], [103, 'Rh_m', 56.11, 'm', 45.6857324275925], [103, 'Ru', 39.25, 'd', 0.01999963216898763], [102, 'Ag', 12.9, 'm', 2513.587135283569], [102, 'Rh', 207.3, 'd', 0.03999986070935975], [101, 'Ag', 11.1, 'm', 79.05852185700594], [101, 'Pd', 8.47, 'h', 15.72779658290452], [101, 'Rh_m', 4.34, 'd', 0.14997505247805204], [100, 'Rh_m', 4.6, 'm', 478.60935671347875], [99, 'Rh_m', 4.7, 'h', 130.58774305687157], [98, 'Rh', 8.72, 'm', 19.563200357626936], [97, 'Tc_m', 91.0, 'd', 0.00999992067339156], [96, 'Tc_m', 51.5, 'm', 24.137840301524626]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=15m'], [106, 'In', 6.2, 'm', 137.05318668696668], [106, 'Ag', 23.96, 'm', 83.05553061227786], [105, 'In', 5.07, 'm', 168.8206301580465], [105, 'Cd', 55.5, 'm', 2528.504133911394], [105, 'Ag_m', 7.23, 'm', 294.758141403612], [104, 'In', 1.8, 'm', 0.2760252558070968], [104, 'Cd', 57.7, 'm', 400.75025006801394], [104, 'Ag_m', 33.5, 'm', 196.9083926955456], [103, 'Ag', 65.7, 'm', 614.2529650734187], [103, 'Pd', 16.99, 'd', 1.3194392745151187], [103, 'Rh_m', 56.11, 'm', 38.6694956682147], [103, 'Ru', 39.25, 'd', 0.019996321994285575], [102, 'Ag', 12.9, 'm', 1217.1089243257286], [102, 'Rh', 207.3, 'd', 0.03999860711542437], [101, 'Ag', 11.1, 'm', 34.033676625297446], [101, 'Pd', 8.47, 'h', 15.440910900073655], [101, 'Rh_m', 4.34, 'd', 0.14975071141139018], [100, 'Rh_m', 4.6, 'm', 62.6197283383252], [99, 'Rh_m', 4.7, 'h', 126.32650053842143], [98, 'Rh', 8.72, 'm', 6.691056796933273], [97, 'Tc_m', 91.0, 'd', 0.009999206762232203], [96, 'Tc_m', 51.5, 'm', 20.128174539103377]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=2.5h'], [106, 'In', 6.2, 'm', 3.8335007145733046e-05], [106, 'Ag', 23.96, 'm', 1.6734321656121036], [105, 'In', 5.07, 'm', 1.635068382021379e-06], [105, 'Cd', 55.5, 'm', 468.580297516029], [105, 'Ag', 41.29, 'd', 0.8684803037665002], [104, 'In', 1.8, 'm', 7.387425668980724e-24], [104, 'Cd', 57.7, 'm', 79.1966977333043], [104, 'Ag', 69.2, 'm', 37.98580813847746], [103, 'Ag', 65.7, 'm', 147.88489516423502], [103, 'Pd', 16.99, 'd', 1.3144034516457397], [103, 'Rh_m', 56.11, 'm', 7.298736237983291], [103, 'Ru', 39.25, 'd', 0.01996325036531761], [102, 'Ag', 12.9, 'm', 0.8623517834741815], [102, 'Rh', 207.3, 'd', 0.039986073336684406], [101, 'Ag', 11.1, 'm', 0.007438932995588576], [101, 'Pd', 8.47, 'h', 12.844678870310881], [101, 'Rh_m', 4.34, 'd', 0.1475256751699028], [100, 'Rh', 20.5, 'h', 18.333260947051503], [99, 'Rh_m', 4.7, 'h', 90.65984269072628], [98, 'Rh', 8.72, 'm', 0.00014656924554411633], [97, 'Tc_m', 91.0, 'd', 0.00999207045324086], [96, 'Tc_m', 51.5, 'm', 3.2723834728944827]]]

In [60]:
z[0]

[['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=0s'],
 [106, 'In_m', 5.2, 'm', 733.53],
 [106, 'Ag', 23.96, 'm', 128.17],
 [105, 'In_m', 48.0, 's', 1312.13],
 [105, 'Cd', 55.5, 'm', 3049.34],
 [105, 'Ag_m', 7.23, 'm', 1241.3],
 [104, 'In', 1.8, 'm', 88.92],
 [104, 'Cd', 57.7, 'm', 479.86],
 [104, 'Ag_m', 33.5, 'm', 268.55],
 [103, 'Ag_m', 5.7, 's', 1534.43],
 [103, 'Pd', 16.99, 'd', 1.32],
 [103, 'Rh_m', 56.11, 'm', 46.54],
 [103, 'Ru', 39.25, 'd', 0.02],
 [102, 'Ag_m', 7.7, 'm', 2824.7],
 [102, 'Rh', 207.3, 'd', 0.04],
 [101, 'Ag_m', 3.1, 's', 88.92],
 [101, 'Pd', 8.47, 'h', 15.76],
 [101, 'Rh_m', 4.34, 'd', 0.15],
 [100, 'Rh_m', 4.6, 'm', 599.96],
 [99, 'Rh_m', 4.7, 'h', 131.07],
 [98, 'Rh_m', 3.6, 'm', 22.23],
 [97, 'Tc_m', 91.0, 'd', 0.01],
 [96, 'Tc_m', 51.5, 'm', 24.63]]

In [65]:
z = [[['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=0s', 'Body_unshield(uSv/h)', 'Body_shield(uSv/h)', 'Skin_unshield(uSv/h)', 'Skin_shield(uSv/h)', 'Breath_Par(uSv/h)'], [106, 'In_m', 5.2, 'm', 733.53, 0.00048046214999999997, 1.3937069999999999e-05, 0.6911319659999999, 0.0011736479999999998, 3.447591e-17], [106, 'Ag', 23.96, 'm', 128.17, 0.00010138246999999999, 2.2173409999999997e-05, 0.13991037199999998, 0.000410144, 7.6902e-18], [105, 'In_m', 48.0, 's', 1312.13, 0.0008594451500000001, 2.4930470000000003e-05, 1.236288886, 0.002099408, 6.167011e-17], [105, 'Cd', 55.5, 'm', 3049.34, 0.0009300487, 0.00069524952, 3.734831632, 0.012502294, 8.538152e-16], [105, 'Ag_m', 7.23, 'm', 1241.3, 4.7169399999999996e-05, 4.7169399999999996e-05, 0.6127056799999999, 0.00260673, 1.86195e-15], [104, 'In', 1.8, 'm', 88.92, 1.316016e-05, 1.2804480000000001e-05, 0.094939884, 0.00023119200000000002, 2.7565200000000006e-18], [104, 'Cd', 57.7, 'm', 479.86, 0.0001463573, 0.00010940808, 0.587732528, 0.001967426, 1.343608e-16], [104, 'Ag_m', 33.5, 'm', 268.55, 1.02049e-05, 1.02049e-05, 0.13255628, 0.000563955, 4.02825e-16], [103, 'Ag_m', 5.7, 's', 1534.43, 5.830834e-05, 5.830834e-05, 0.757394648, 0.003222303, 2.301645e-15], [103, 'Pd', 16.99, 'd', 1.32, 6.5999999999999995e-09, 6.5999999999999995e-09, 0.0017394959999999999, 5.28e-07, 7.259999999999999e-19], [103, 'Rh_m', 56.11, 'm', 46.54, 2.07103e-05, 2.07103e-05, 0.068679078, 0.000456092, 7.446399999999999e-18], [103, 'Ru', 39.25, 'd', 0.02, 2.6000000000000003e-10, 2.6000000000000003e-10, 2.9528000000000003e-05, 3.0000000000000004e-08, 1.4e-19], [102, 'Ag_m', 7.7, 'm', 2824.7, 0.0001073386, 0.0001073386, 1.39427192, 0.00593187, 4.23705e-15], [102, 'Rh', 207.3, 'd', 0.04, 2.36e-09, 2.2000000000000003e-09, 4.5836000000000006e-05, 1.48e-07, 9.600000000000002e-22], [101, 'Ag_m', 3.1, 's', 88.92, 3.3789600000000003e-06, 3.3789600000000003e-06, 0.043890912000000004, 0.000186732, 1.3338e-16], [101, 'Pd', 8.47, 'h', 15.76, 7.88e-08, 7.88e-08, 0.020768528, 6.304e-06, 8.667999999999999e-18], [101, 'Rh_m', 4.34, 'd', 0.15, 6.675e-08, 6.675e-08, 0.000221355, 1.47e-06, 2.4e-20], [100, 'Rh_m', 4.6, 'm', 599.96, 0.00026698220000000004, 0.00026698220000000004, 0.8853609720000001, 0.005879608000000001, 9.59936e-17], [99, 'Rh_m', 4.7, 'h', 131.07, 5.832615e-05, 5.832615e-05, 0.19341999899999998, 0.001284486, 2.0971199999999998e-17], [98, 'Rh_m', 3.6, 'm', 22.23, 9.89235e-06, 9.89235e-06, 0.032804811, 0.00021785400000000002, 3.5568e-18], [97, 'Tc_m', 91.0, 'd', 0.01, 3.740000000000001e-09, 3.5600000000000005e-09, 1.1476000000000002e-05, 6.4e-08, 2.2000000000000005e-22], [96, 'Tc_m', 51.5, 'm', 24.63, 9.21162e-06, 8.768279999999999e-06, 0.028265388, 0.000157632, 5.418600000000001e-19], ['sum', '', '', '', '', 0.0031225372599999993, 0.0014697406800000003, 10.657001175000001, 0.03889991800000001, 1.016968778e-14]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=1.5m', 'Body_unshield(uSv/h)', 'Body_shield(uSv/h)', 'Skin_unshield(uSv/h)', 'Skin_shield(uSv/h)', 'Breath_Par(uSv/h)'], [106, 'In', 6.2, 'm', 619.7527990880288, 9.172341426502826e-05, 8.924440306867615e-05, 0.6617100635862884, 0.0016113572776288749, 1.9212336771728894e-17], [106, 'Ag', 23.96, 'm', 122.72826695650605, 9.707805916259628e-05, 2.1231990183475548e-05, 0.13397017620972201, 0.0003927304542608194, 7.363696017390362e-18], [105, 'In', 5.07, 'm', 1068.5982944471589, 0.0001581525475781795, 0.00015387815440039086, 1.1409423989812315, 0.002778355565562613, 3.3126547127861925e-17], [105, 'Cd', 55.5, 'm', 2992.758220739961, 0.000912791257325688, 0.000682348874328711, 3.665530268762304, 0.01227030870503384, 8.379723018071891e-16], [105, 'Ag_m', 7.23, 'm', 1075.0673211343258, 4.085255820310437e-05, 4.085255820310437e-05, 0.5306532297119032, 0.002257641374382084, 1.6126009817014886e-15], [104, 'In', 1.8, 'm', 49.91078387337419, 7.38679601325938e-06, 7.187152877765883e-06, 0.053289743941601624, 0.0001297680380707729, 1.5472343000746001e-18], [104, 'Cd', 57.7, 'm', 471.29244174818865, 0.00014374419473319754, 0.00010745467671858702, 0.5772389826531814, 0.0019322990111675735, 1.3196188368949282e-16], [104, 'Ag_m', 33.5, 'm', 260.34489354421373, 9.893105954680122e-06, 9.893105954680122e-06, 0.1285062394534239, 0.0005467242764428488, 3.9051734031632057e-16], [103, 'Ag', 65.7, 'm', 708.2549295060942, 0.0005602296492393205, 0.0001225281028045543, 0.7731310810488524, 0.0022664157744195015, 4.249529577036565e-17], [103, 'Pd', 16.99, 'd', 1.31994391673, 6.59971958365e-09, 6.59971958365e-09, 0.001739422093466794, 5.27977566692e-07, 7.259691542014999e-19], [103, 'Rh_m', 56.11, 'm', 45.6857324275925, 2.033015093027866e-05, 2.033015093027866e-05, 0.06741843534339825, 0.0004477201777904065, 7.309717188414799e-18], [103, 'Ru', 39.25, 'd', 0.01999963216898763, 2.5999521819683915e-10, 2.5999521819683915e-10, 2.9527456934293334e-05, 2.9999448253481445e-08, 1.3999742518291339e-19], [102, 'Ag', 12.9, 'm', 2513.587135283569, 0.0019882474240093028, 0.0004348505744040574, 2.743831716875544, 0.00804347883290742, 1.5081522811701414e-16], [102, 'Rh', 207.3, 'd', 0.03999986070935975, 2.3599917818522252e-09, 2.199992339014786e-09, 4.583584038685533e-05, 1.4799948462463106e-07, 9.59996657024634e-22], [101, 'Ag', 11.1, 'm', 79.05852185700594, 6.25352907888917e-05, 1.3677124281262027e-05, 0.08630028245910769, 0.00025298726994241904, 4.743511311420356e-18], [101, 'Pd', 8.47, 'h', 15.72779658290452, 7.863898291452261e-08, 7.863898291452261e-08, 0.020726090336951577, 6.291118633161809e-06, 8.650288120597486e-18], [101, 'Rh_m', 4.34, 'd', 0.14997505247805204, 6.673889835273316e-08, 6.673889835273316e-08, 0.0002213181849418614, 1.46975551428491e-06, 2.3996008396488327e-20], [100, 'Rh_m', 4.6, 'm', 478.60935671347875, 0.00021298116373749803, 0.00021298116373749803, 0.7062838277020805, 0.004690371695792092, 7.65774970741566e-17], [99, 'Rh_m', 4.7, 'h', 130.58774305687157, 5.8111545660307845e-05, 5.8111545660307845e-05, 0.19270833242902538, 0.0012797598819573412, 2.0894038889099448e-17], [98, 'Rh', 8.72, 'm', 19.563200357626936, 1.1542288210999892e-06, 1.0759760196694816e-06, 0.02241747128980471, 7.238384132321967e-05, 4.695168085830465e-19], [97, 'Tc_m', 91.0, 'd', 0.00999992067339156, 3.739970331848443e-09, 3.5599717597273952e-09, 1.1475908964784154e-05, 6.399949230970598e-08, 2.199982548146143e-22], [96, 'Tc_m', 51.5, 'm', 24.137840301524626, 9.02755227277021e-06, 8.593071147342767e-06, 0.027700585530029662, 0.00015448217792975763, 5.310324866335418e-19], ['sum', '', '', '', '', 0.004374397276253386, 0.0019843966222805294, 11.534406505799145, 0.039135315204750915, 3.3476795900805246e-15]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=15m', 'Body_unshield(uSv/h)', 'Body_shield(uSv/h)', 'Skin_unshield(uSv/h)', 'Skin_shield(uSv/h)', 'Breath_Par(uSv/h)'], [106, 'In', 6.2, 'm', 137.05318668696668, 2.028387162967107e-05, 1.9735658882923204e-05, 0.14633168742567432, 0.0003563382853861134, 4.248648787295967e-18], [106, 'Ag', 23.96, 'm', 83.05553061227786, 6.569692471431179e-05, 1.436860679592407e-05, 0.09066341721636251, 0.00026577769795928913, 4.9833318367366715e-18], [105, 'In', 5.07, 'm', 168.8206301580465, 2.4985453263390883e-05, 2.4310170742758696e-05, 0.18024978681974627, 0.00043893363841092094, 5.233439534899442e-18], [105, 'Cd', 55.5, 'm', 2528.504133911394, 0.0007711937608429752, 0.0005764989425317978, 3.0969118632146753, 0.010366866949036716, 7.079811574951903e-16], [105, 'Ag_m', 7.23, 'm', 294.758141403612, 1.1200809373337257e-05, 1.1200809373337257e-05, 0.1454926185968229, 0.0006189920969475852, 4.42137212105418e-16], [104, 'In', 1.8, 'm', 0.2760252558070968, 4.0851737859450324e-08, 3.974763683622194e-08, 0.00029471216562523725, 7.176656650984517e-07, 8.556782930020001e-21], [104, 'Cd', 57.7, 'm', 400.75025006801394, 0.00012222882627074426, 9.137105701550717e-05, 0.49083890628330346, 0.0016430760252788572, 1.122100700190439e-16], [104, 'Ag_m', 33.5, 'm', 196.9083926955456, 7.482518922430732e-06, 7.482518922430732e-06, 0.0971939826345213, 0.0004135076246606457, 2.9536258904331836e-16], [103, 'Ag', 65.7, 'm', 614.2529650734187, 0.0004858740953730742, 0.00010626576295770144, 0.6705185366741439, 0.00196560948823494, 3.6855177904405126e-17], [103, 'Pd', 16.99, 'd', 1.3194392745151187, 6.597196372575594e-09, 6.597196372575594e-09, 0.0017387570759560234, 5.277757098060475e-07, 7.256916009833152e-19], [103, 'Rh_m', 56.11, 'm', 38.6694956682147, 1.720792557235554e-05, 1.720792557235554e-05, 0.05706457475758443, 0.0003789610575485041, 6.187119306914352e-18], [103, 'Ru', 39.25, 'd', 0.019996321994285575, 2.599521859257125e-10, 2.599521859257125e-10, 2.9522569792363224e-05, 2.9994482991428366e-08, 1.3997425395999903e-19], [102, 'Ag', 12.9, 'm', 1217.1089243257286, 0.0009627331591416513, 0.00021055984390835105, 1.3285961017939654, 0.0038947485578423313, 7.302653545954372e-17], [102, 'Rh', 207.3, 'd', 0.03999860711542437, 2.359917819810038e-09, 2.1999233913483404e-09, 4.583440389356479e-05, 1.4799484632707017e-07, 9.59966570770185e-22], [101, 'Ag', 11.1, 'm', 34.033676625297446, 2.692063821061028e-05, 5.8878260561764576e-06, 0.03715116140417469, 0.00010890776520095183, 2.0420205975178465e-18], [101, 'Pd', 8.47, 'h', 15.440910900073655, 7.720455450036827e-08, 7.720455450036827e-08, 0.020348032384117063, 6.176364360029462e-06, 8.49250099504051e-18], [101, 'Rh_m', 4.34, 'd', 0.14975071141139018, 6.663906657806864e-08, 6.663906657806864e-08, 0.0002209871248297885, 1.467556971831624e-06, 2.396011382582243e-20], [100, 'Rh_m', 4.6, 'm', 62.6197283383252, 2.7865779110554716e-05, 2.7865779110554716e-05, 0.0924079331088665, 0.000613673337715587, 1.0019156534132032e-17], [99, 'Rh_m', 4.7, 'h', 126.32650053842143, 5.621529273959753e-05, 5.621529273959753e-05, 0.18642001684454848, 0.00123799970527653, 2.0212240086147426e-17], [98, 'Rh', 8.72, 'm', 6.691056796933273, 3.947723510190631e-07, 3.6800812383133e-07, 0.007667281983605837, 2.4756910148653107e-05, 1.6058536312639855e-19], [97, 'Tc_m', 91.0, 'd', 0.009999206762232203, 3.739703329074844e-09, 3.5597176073546645e-09, 1.1475089680337676e-05, 6.399492327828611e-08, 2.1998254876910847e-22], [96, 'Tc_m', 51.5, 'm', 20.128174539103377, 7.527937277624663e-06, 7.165630135920802e-06, 0.023099093101075035, 0.00012882031705026162, 4.428198398602743e-19], ['sum', '', '', '', '', 0.0026080094169219936, 0.0011767000409166394, 6.673296282672965, 0.022466100803657248, 1.7304939676094092e-15]], [['A', 'Isotope', 't1/2', 'unit', 'Prodx_A(Bq)@cd=2.5h', 'Body_unshield(uSv/h)', 'Body_shield(uSv/h)', 'Skin_unshield(uSv/h)', 'Skin_shield(uSv/h)', 'Breath_Par(uSv/h)'], [106, 'In', 6.2, 'm', 3.8335007145733046e-05, 5.673581057568491e-12, 5.520241028985559e-12, 4.0930287129499174e-08, 9.967101857890593e-11, 1.1883852215177245e-24], [106, 'Ag', 23.96, 'm', 1.6734321656121036, 1.323684842999174e-06, 2.8950376465089393e-07, 0.0018267185519821724, 5.354982929958732e-06, 1.0040592993672622e-19], [105, 'In', 5.07, 'm', 1.635068382021379e-06, 2.419901205391641e-13, 2.3544984701107857e-13, 1.7457625114842263e-09, 4.251177793255586e-12, 5.0687119842662757e-26], [105, 'Cd', 55.5, 'm', 468.580297516029, 0.00014291699074238886, 0.00010683630783365462, 0.5739171483976324, 0.001921179219815719, 1.3120248330448813e-16], [105, 'Ag', 41.29, 'd', 0.8684803037665002, 6.869679202793016e-07, 1.5024709255160453e-07, 0.0009480330995915116, 2.7791369720528007e-06, 5.210881822599001e-20], [104, 'In', 1.8, 'm', 7.387425668980724e-24, 1.0933389990091471e-30, 1.0637892963332241e-30, 7.887554386770718e-27, 1.9207306739349882e-29, 2.2901019573840246e-43], [104, 'Cd', 57.7, 'm', 79.1966977333043, 2.415499280865781e-05, 1.805684708319338e-05, 0.09700011538375111, 0.00032470646070654764, 2.2175075365325204e-17], [104, 'Ag', 69.2, 'm', 37.98580813847746, 3.004677423753567e-05, 6.5715448079566e-06, 0.041465308163961996, 0.00012155458604312787, 2.2791484883086476e-18], [103, 'Ag', 65.7, 'm', 147.88489516423502, 0.0001169769520749099, 2.558408686341266e-05, 0.16143115156127896, 0.00047323166452555205, 8.873093709854102e-18], [103, 'Pd', 16.99, 'd', 1.3144034516457397, 6.572017258228699e-09, 6.572017258228699e-09, 0.0017321208685787558, 5.257613806582959e-07, 7.229218984051568e-19], [103, 'Rh_m', 56.11, 'm', 7.298736237983291, 3.2479376259025643e-06, 3.2479376259025643e-06, 0.010770745066391942, 7.152761513223625e-05, 1.1677977980773264e-18], [103, 'Ru', 39.25, 'd', 0.01996325036531761, 2.595222547491289e-10, 2.595222547491289e-10, 2.947374283935492e-05, 2.9944875547976415e-08, 1.3974275255722327e-19], [102, 'Ag', 12.9, 'm', 0.8623517834741815, 6.821202607280776e-07, 1.4918685854103341e-07, 0.0009413432068404165, 2.7595257071173807e-06, 5.174110700845089e-20], [102, 'Rh', 207.3, 'd', 0.039986073336684406, 2.35917832686438e-09, 2.1992340335176422e-09, 4.582004143650666e-05, 1.479484713457323e-07, 9.596657600804258e-22], [101, 'Ag', 11.1, 'm', 0.007438932995588576, 5.8841959995105626e-09, 1.2869354082368235e-09, 8.120339257984488e-06, 2.380458558588344e-08, 4.463359797353145e-22], [101, 'Pd', 8.47, 'h', 12.844678870310881, 6.422339435155441e-08, 6.422339435155441e-08, 0.01692671781529568, 5.137871548124353e-06, 7.064573378670985e-18], [101, 'Rh_m', 4.34, 'd', 0.1475256751699028, 6.564892545060676e-08, 6.564892545060676e-08, 0.0002177036388482256, 1.4457516166650475e-06, 2.360410802718445e-20], [100, 'Rh', 20.5, 'h', 18.333260947051503, 1.0816623958760388e-06, 1.0083293520878328e-06, 0.021008083719226318, 6.783306550409057e-05, 4.399982627292361e-19], [99, 'Rh_m', 4.7, 'h', 90.65984269072628, 4.0343629997373195e-05, 4.0343629997373195e-05, 0.13378672985870477, 0.0008884664583691176, 1.4505574830516204e-17], [98, 'Rh', 8.72, 'm', 0.00014656924554411633, 8.647585487102863e-12, 8.061308504926397e-12, 1.679536984690029e-07, 5.423062085132304e-10, 3.517661893058792e-24], [97, 'Tc_m', 91.0, 'd', 0.00999207045324086, 3.737034349512082e-09, 3.557177081353746e-09, 1.1466900052139212e-05, 6.39492509007415e-08, 2.1982554997129894e-22], [96, 'Tc_m', 51.5, 'm', 3.2723834728944827, 1.2238714188625365e-06, 1.1649685163504357e-06, 0.0037553872734937084, 2.0943254226524688e-05, 7.199243640367862e-20], ['sum', '', '', '', '', 0.00036283428315666085, 0.00020354635081851246, 1.065822398258912, 0.003907711647889277, 1.8887189277255835e-16]]]

In [66]:
z[1]

[['A',
  'Isotope',
  't1/2',
  'unit',
  'Prodx_A(Bq)@cd=1.5m',
  'Body_unshield(uSv/h)',
  'Body_shield(uSv/h)',
  'Skin_unshield(uSv/h)',
  'Skin_shield(uSv/h)',
  'Breath_Par(uSv/h)'],
 [106,
  'In',
  6.2,
  'm',
  619.7527990880288,
  9.172341426502826e-05,
  8.924440306867615e-05,
  0.6617100635862884,
  0.0016113572776288749,
  1.9212336771728894e-17],
 [106,
  'Ag',
  23.96,
  'm',
  122.72826695650605,
  9.707805916259628e-05,
  2.1231990183475548e-05,
  0.13397017620972201,
  0.0003927304542608194,
  7.363696017390362e-18],
 [105,
  'In',
  5.07,
  'm',
  1068.5982944471589,
  0.0001581525475781795,
  0.00015387815440039086,
  1.1409423989812315,
  0.002778355565562613,
  3.3126547127861925e-17],
 [105,
  'Cd',
  55.5,
  'm',
  2992.758220739961,
  0.000912791257325688,
  0.000682348874328711,
  3.665530268762304,
  0.01227030870503384,
  8.379723018071891e-16],
 [105,
  'Ag_m',
  7.23,
  'm',
  1075.0673211343258,
  4.085255820310437e-05,
  4.085255820310437e-05,
  0.530653